# Indirect Zero-Noise-Extrapolation
---

This notebook contains the experimental data and Zero-Noise-Extrapolation using various method.

## Zero-Noise-Extrapolation code

In [1]:

import itertools
from collections import Counter
import numpy as np

def get_monomials(n: int, d: int) -> list[str]:

    """
    Compute monomials of degree `d` in graded lexicographical order.
    """

    variables = [f"λ_{i}" for i in range(1, n + 1)]
    
    monomials = []
    for degree in range(d, -1, -1):
        # Generate combinations for the current degree
        combos = list(itertools.combinations_with_replacement(variables, degree))
        
        # Sort combinations lexicographically
        combos.sort()
        
        # Construct monomials from sorted combinations
        for combo in combos:
            monomial_parts = []
            counts = Counter(combo)
            # Ensure variables are processed in lexicographical order
            for var in sorted(counts.keys()):
                count = counts[var]
                if count > 1:
                    monomial_parts.append(f"{var}**{count}")
                else:
                    monomial_parts.append(var)
            monomial = "*".join(monomial_parts)
            # Handle the case where degree is 0
            if not monomial:
                monomial = "1"
            monomials.append(monomial)
    # "1" should be the first monomial. Note that order d > c > b > a means vector of monomials = [a, b, c, d].            
    return monomials[::-1]

def sample_matrix(sample_points: list[int], degree: int) -> np.ndarray:

    """
    Construct a matrix from monomials evaluated at sample points.
    """
    n = len(sample_points[0])  # type: ignore # Number of variables based on the first sample point
    monomials = get_monomials(n, degree) # type: ignore
    matrix = np.zeros((len(sample_points), len(monomials)))

    for i, point in enumerate(sample_points):
        for j, monomial in enumerate(monomials):
            var_mapping = {f"λ_{k+1}": point[k] for k in range(n)} # type: ignore
            matrix[i, j] = eval(monomial, {}, var_mapping)
    return matrix

def generate_modified_matrices(matrix):

    n = len(matrix)  # Size of the square matrix
    identity_row = [1] + [0] * (n - 1)  # Row to replace with

    modified_matrices = []
    determinants = []
    for i in range(n):
        # Create a copy of the original matrix
        modified_matrix = np.copy(matrix)
        # Replace the i-th row with the identity_row
        modified_matrix[i] = identity_row
        modified_matrices.append(modified_matrix)
        # Calculate the determinant of the modified matrix
        determinant = np.linalg.det(modified_matrix)
        determinants.append(determinant)
    
    return modified_matrices 

class ZeroNoiseExtrapolation:
    def __init__(self, dataPoints: list[tuple[float]], degree: int):
        """
        Initialize with a list of data points, each represented as a tuple of floats.
        """
        self.dataPoints = dataPoints
        self.degree = degree

        self.NoiseData = [tuple(point[:3]) for point in self.dataPoints]
        self.ExpectationVals = [point[-1] for point in dataPoints]
  
    def getRichardsonZNE(self):

        RichardsonZNEval = 0

        sampleMatrix = sample_matrix(sample_points = self.NoiseData, degree = self.degree) # type: ignore
        detA = np.linalg.det(sampleMatrix)

        matrices = generate_modified_matrices(sampleMatrix) # type: ignore

        for E, matrix in zip(self.ExpectationVals, matrices):
            RichardsonZNEval += E * (np.linag.det(matrix)/detA)
        
        return RichardsonZNEval
    
    

In [2]:
data_points = [
    (12, 5, 18, -3.717054761326808),
    (20, 1, 30, -3.874319285474221),
    (4, 5, 6, -3.8215866373279432),
    (28, 7, 6, -3.687171847866942),
    (4, 9, 18, -3.5663020157522007),
    (4, 1, 0, -3.9308629429508755),
    (12, 3, 6, -3.84785786134227),
    (4, 11, 6, -3.6634401626134006),
    (20, 11, 30, -3.2597725821448074),
    (36, 11, 42, -3.053170929882078)
]
ZNE = ZeroNoiseExtrapolation(dataPoints = data_points, degree = 2)
ZNE.getRichardsonZNE()

AttributeError: module 'numpy' has no attribute 'linag'

### Linear regression

The general single variable linear regression: $E_{linear}(\lambda)=c_0 + c_1\lambda$

Multivariate linear regression:

$$E_{linear}(n_R, n_T, nY)=c_0 + c_1 n_R + c_2 n_T + c_3 n_Y$$

Where independent variables are: $n_R, n_T, n_Y$

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import itertools
import os
import pickle
import random
from collections import Counter, defaultdict
from copy import deepcopy
from typing import Any, Callable, Generator, Optional
# Functional programming utilities.
import functools

from sympy import *
init_printing(use_unicode=True)

In [2]:
def get_monomials(n: int, d: int) -> list[str]:
    """Compute monomials of degree `d` in graded lexicographical order."""
    variables = [f"λ_{i}" for i in range(1, n + 1)]
    
    monomials = []
    for degree in range(d, -1, -1):
        # Generate combinations for the current degree
        combos = list(itertools.combinations_with_replacement(variables, degree))
        
        # Sort combinations lexicographically
        combos.sort()
        
        # Construct monomials from sorted combinations
        for combo in combos:
            monomial_parts = []
            counts = Counter(combo)
            # Ensure variables are processed in lexicographical order
            for var in sorted(counts.keys()):
                count = counts[var]
                if count > 1:
                    monomial_parts.append(f"{var}**{count}")
                else:
                    monomial_parts.append(var)
            monomial = "*".join(monomial_parts)
            # Handle the case where degree is 0
            if not monomial:
                monomial = "1"
            monomials.append(monomial)
    # "1" should be the first monomial. Note that order d > c > b > a means vector of monomials = [a, b, c, d].            
    return monomials[::-1]

def sample_matrix(sample_points: list[int], degree: int) -> np.ndarray:
    """Construct a matrix from monomials evaluated at sample points."""
    n = len(sample_points[0])  # Number of variables based on the first sample point
    monomials = get_monomials(n, degree)
    matrix = np.zeros((len(sample_points), len(monomials)))

    for i, point in enumerate(sample_points):
        for j, monomial in enumerate(monomials):
            var_mapping = {f"λ_{k+1}": point[k] for k in range(n)}
            matrix[i, j] = eval(monomial, {}, var_mapping)
    return matrix

def get_eta_coeffs_from_sample_matrix(mat: np.ndarray) -> list[float]:
    """Given a sample matrix compute the eta coefficients."""
    n_rows, n_cols = mat.shape
    if n_rows != n_cols:
        raise ValueError("The matrix must be square.")

    det_m = np.linalg.det(mat)    
    if det_m == 0:
        raise ValueError("The matrix is singular.")
    
    terms = []
    for i in range(n_rows):
        new_mat = mat.copy()
        new_mat[i] = np.array([[0] * (n_cols - 1) + [1]])        
        terms.append(np.linalg.det(new_mat) / det_m)

    return terms

def get_eta_coeffs(lre_scale_factors: list[float], degree: int) -> list[float]:
    """Obtain eta coefficients from list of scale factors and degree."""
    mat = sample_matrix(sample_points=lre_scale_factors, degree=degree)
    return get_eta_coeffs_from_sample_matrix(mat)

def get_eta_coeffs_single_variable(scale_factors: list[float]) -> list[float]:
    """Returns the array of single-variable Richardson extrapolation coefficients associated
    to the input array of scale factors."""
    # Lagrange interpolation formula.
    richardson_coeffs = []
    for l in scale_factors:
        coeff = 1.0
        for l_prime in scale_factors:
            if l_prime == l:
                continue
            coeff *= l_prime / (l_prime - l)
        richardson_coeffs.append(coeff)

    return richardson_coeffs

In [5]:
# sample_points = [ (4, 1, 0), (12, 3, 1), (20, 5, 2), (28, 7, 3), (36, 9, 4), (44, 11, 5), (52, 13, 6), (60, 15, 7), (68, 17, 8), (76, 19, 9), (84, 21, 10), (92, 23, 11), (100, 25, 12), (108, 27, 13), (116, 29, 14), (124, 31, 15), (132, 33, 16), (140, 35, 17), (148, 37, 18), (156, 39, 19)]
# sample_points = [(4, 1, -5.907748909320789, 0),
# (12, 1, -5.916920204491979, 0),
# (12, 3, -5.800746641641066, 1),
# (20, 3, -5.764809805003449, 1),
# (20, 5, -5.679828979633399, 2)
# ] 

In [6]:
n, d = 2, 1
sample_points = [
    (4, 1), (12, 5), (20, 5)
]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")

Monomials: ['1', 'λ_2', 'λ_1']
Monomials: 3


In [7]:
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

⎡1.0  1.0  4.0 ⎤
⎢              ⎥
⎢1.0  5.0  12.0⎥
⎢              ⎥
⎣1.0  5.0  20.0⎦

In [8]:
m = np.linalg.det(mat)

In [9]:
m

In [10]:
from sympy import Matrix

# Define the matrix
m1 = Matrix([
    [1, Symbol('λ1'), Symbol('λ2')],
    [1, 12, 5],
    [1, 20, 5]
])

# Calculate determinant
det1 = m1.det()

print(det1/m)


0.25*λ2 - 1.25


In [11]:
# Define the matrix
m2 = Matrix([
    [1, 4, 1],
    [1, Symbol('λ1'), Symbol('λ2')],
    [1, 20, 5]
])

# Calculate determinant
det2 = m2.det()

print(det2/m)

0.125*λ1 - 0.5*λ2


In [12]:
m3 = Matrix([
    [1, 4, 1],
    [1, 12, 5],
    [1, Symbol('λ1'), Symbol('λ2')]
])

# Calculate determinant
det3 = m3.det()

print(det3/m)

-0.125*λ1 + 0.25*λ2 + 0.25


In [13]:
print((det1/32)+(det2/32)+(det3/32))

-1


In [14]:
n =2
d= 1
sample_points = [(0,0), (0,1), (1,1)]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

Monomials: ['1', 'λ_2', 'λ_1']
Monomials: 3


⎡1.0   0    0 ⎤
⎢             ⎥
⎢1.0  1.0   0 ⎥
⎢             ⎥
⎣1.0  1.0  1.0⎦

In [15]:
get_eta_coeffs_from_sample_matrix(mat)

In [16]:
n, d = 2, 2
sample_points = [
    (4, 1), (12, 7), (36, 5), (36, 13), (52, 5), (52, 13)
]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")



Monomials: ['1', 'λ_2', 'λ_1', 'λ_2**2', 'λ_1*λ_2', 'λ_1**2']
Monomials: 6


In [17]:
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

⎡1.0  1.0   4.0    1.0    4.0    16.0 ⎤
⎢                                     ⎥
⎢1.0  7.0   12.0  49.0   84.0   144.0 ⎥
⎢                                     ⎥
⎢1.0  5.0   36.0  25.0   180.0  1296.0⎥
⎢                                     ⎥
⎢1.0  13.0  36.0  169.0  468.0  1296.0⎥
⎢                                     ⎥
⎢1.0  5.0   52.0  25.0   260.0  2704.0⎥
⎢                                     ⎥
⎣1.0  13.0  52.0  169.0  676.0  2704.0⎦

In [18]:
A = np.linalg.det(mat)

In [19]:
get_eta_coeffs_from_sample_matrix(mat)

$ E = c_1 + c_2 n_T + c_3 n_R + c_4 n^{2}_{T} + c_5 n_R n_T + c_6 n^{2}_T$

In [20]:
(4, 1, -5.942817754597376), (12, 7, 5.610630688805676), (36, 5, -5.61959508599682), (36, 13, -5.267674373577144), (52, 5, -5.540521346789942), (52, 13, -5.135264624148199)

In [21]:
A

In [22]:
mat

array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

In [23]:
A = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M1 = np.array([[1, 0, 0, 0, 0, 0],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])



M2 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M3 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M4 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M5 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M6 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1, 0, 0, 0, 0, 0]])

In [24]:
Matrix(M6)

⎡1.0  1.0   4.0    1.0    4.0    16.0 ⎤
⎢                                     ⎥
⎢1.0  7.0   12.0  49.0   84.0   144.0 ⎥
⎢                                     ⎥
⎢1.0  5.0   36.0  25.0   180.0  1296.0⎥
⎢                                     ⎥
⎢1.0  13.0  36.0  169.0  468.0  1296.0⎥
⎢                                     ⎥
⎢1.0  5.0   52.0  25.0   260.0  2704.0⎥
⎢                                     ⎥
⎣1.0   0     0      0      0      0   ⎦

In [25]:
detA = np.linalg.det(mat)

In [26]:
detA

In [27]:
RE = (-5.942817754597376 * np.linalg.det(M1)/detA) + (-5.610630688805676*np.linalg.det(M2)/detA) + (-5.61959508599682 * np.linalg.det(M3)/detA) + (-5.267674373577144  * np.linalg.det(M4)/detA) + (-5.540521346789942 * np.linalg.det(M5)/detA) + (-5.135264624148199 * np.linalg.det(M6)/detA)

In [28]:
RE

In [29]:
a = -6.00752391934362
b = -6.026674183332272

In [30]:
b<a

True

In [31]:
import numpy as np

# Coefficients and matrices
coefficients = [-5.942817754597376, -5.610630688805676, -5.61959508599682, -5.267674373577144, -5.540521346789942, -5.135264624148199]
matrices = [M1, M2, M3, M4, M5, M6]  # Assuming M1, M2, ..., M6 are defined

# Calculate detA
detA = np.linalg.det(A)  # Assuming A is defined

# Initialize result
RE = 0

# Loop over coefficients and matrices
for coeff, matrix in zip(coefficients, matrices):
    RE += coeff * np.linalg.det(matrix) / detA

print("Result:", RE)


Result: -6.007523919343623


In [32]:
import numpy as np

param = np.array([1.40837213e+00, 6.28293547e+00, 8.62288600e-04, 3.14127596e+00,
                  9.42500283e+00])

# Open a file in write mode
with open('output.txt', 'w') as file:
    # Join the array elements into a single string with commas
    formatted_param = ', '.join(map(str, param))
    file.write(f"[{formatted_param}]\n")


In [33]:
type(param)

numpy.ndarray

In [34]:
param = [1.40837213e+00, 6.28293547e+00, 8.62288600e-04, 3.14127596e+00, 9.42500283e+00]

# Join the list elements into a single string with commas
formatted_param = ', '.join(map(str, param))

# Print the formatted string
print(formatted_param)


1.40837213, 6.28293547, 0.0008622886, 3.14127596, 9.42500283


In [35]:
n, d = 3, 1
sample_points = [
    (4, 1), (12, 7), (36, 5), (36, 13), (52, 5), (52, 13)
]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")

Monomials: ['1', 'λ_3', 'λ_2', 'λ_1']
Monomials: 4


In [36]:
n, d = 3, 2

data_points = [
    [4, 1, [-5.937502025309547]],
    # [12, 3, 3, [-5.735146258695903]],
    [28, 5,  [-5.5437873408259275]],
    [20, 7,  [-5.274649724774608]],
    [20, 7, [-5.581262029732842]],
    [36, 3, [-5.668152907726871]],
    [36, 5, [-5.597490245829486]]

]

In [37]:
# Extract the first three values from each sublist and store them in an array of tuples
noise_data = [tuple(point[:2]) for point in data_points]

# Extract the last value from each sublist and store them in a list
expectation_values = [point[-1][0] for point in data_points]

# Print the resulting array of tuples
print(noise_data)
print(expectation_values)



[(4, 1), (28, 5), (20, 7), (20, 7), (36, 3), (36, 5)]
[-5.937502025309547, -5.5437873408259275, -5.274649724774608, -5.581262029732842, -5.668152907726871, -5.597490245829486]


In [38]:
A = sample_matrix(sample_points=noise_data2, degree=d)

NameError: name 'noise_data2' is not defined

In [39]:
A

array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

In [40]:
detA = np.linalg.det(A)

In [41]:
detA

In [42]:
import numpy as np

def generate_modified_matrices(matrix):
    n = len(matrix)  # Size of the square matrix
    identity_row = [1] + [0] * (n - 1)  # Row to replace with

    modified_matrices = []
    determinants = []
    for i in range(n):
        # Create a copy of the original matrix
        modified_matrix = np.copy(matrix)
        # Replace the i-th row with the identity_row
        modified_matrix[i] = identity_row
        modified_matrices.append(modified_matrix)
        # Calculate the determinant of the modified matrix
        determinant = np.linalg.det(modified_matrix)
        determinants.append(determinant)
    
    return modified_matrices 

#matrices = generate_modified_matrices(A)


In [43]:
# Loop over coefficients and matrices
for coeff, matrix in zip(coefficients, matrices):
    RE += coeff * np.linalg.det(matrix) / detA

In [44]:
RE

In [45]:
A

array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

In [46]:
matrices

[array([[1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
        [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
        [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
        [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
        [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]]),
 array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
        [1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
        [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
        [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
        [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]]),
 array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
        

---

```yml
nqubits: 6
state: "DMatrix"
layer: 10
iteration: 1
Exact:  -7.29622981055876
```

In [47]:
noise_data = [(4, 1), (12, 5), (28, 5), (44, 7), (20, 7), (44, 9) ]
e_val = [-6.653015381479125, -5.944264589467563, -5.953575981631173, -5.872561179170413, -6.1660998573827825, -5.435931276419277]
n, d = 2, 2
monomials = get_monomials(n=n, d=d)
# print(f"Monomials: {monomials}")
# print(f"Monomials: {len(monomials)}")

In [48]:
# This is the sample matrix
A = sample_matrix(sample_points= noise_data, degree= d)

In [49]:
detA = np.linalg.det(A)

In [50]:
matrices = generate_modified_matrices(A)
RE = 0

In [51]:
# Loop over coefficients and matrices
for E, matrix in zip(e_val, matrices):
    RE += E * np.linalg.det(matrix) / detA

In [52]:
RE

```yml
nqubits: 5
state: "DMatrix"
layer: 10
iteration: 1
optimizer: "SLSQP"
constraint: False
etime: True
Exact: -6.026674183332272
```

In [53]:
n, d = 3, 2
detA = 0, 0

data_points = [

    [36, 9, 66, [-4.331378173828132]],
    [12, 5, 66, [-5.137601571314731]],
    [60, 15, 18, [-4.695641935081705]],
    [4, 1, 0, [-5.847083978239625]],
    [36, 9, 18, [-5.163901197964829]],
    [36, 9, 42, [-4.750859450731831]],
    [36, 9, 6, [-5.359889992461941]],
    [12, 3, 6, [-5.7587171216700765]],
    [12, 5, 42, [-5.381585427158724]],
    [20, 7, 18, [-5.366706940188518]]
]

noise_data = [tuple(point[:3]) for point in data_points]
e_vals = [point[-1][0] for point in data_points]


# noise_data = [(4, 1), (12, 5), (28, 5), (44, 7), (20, 7), (44, 9) ]
# e_val = [-6.653015381479125, -5.944264589467563, -5.953575981631173, -5.872561179170413, -6.1660998573827825, -5.435931276419277]

monomials = get_monomials(n=n, d=d)
# print(f"Monomials: {monomials}")
# print(f"Monomials: {len(monomials)}")

A = sample_matrix(sample_points=noise_data, degree= d)
detA = np.linalg.det(A)

matrices = generate_modified_matrices(A)

RE = 0
# Loop over coefficients and matrices
for E, matrix in zip(e_vals, matrices):
    print(np.linalg.det(matrix) / detA)
    RE += E * (np.linalg.det(matrix) / detA )

RE

0.03339843750000001
0.0
0.0
0.28125000000000105
-0.5677083333333373
0.09027777777777846
-0.075
0.7500000000000061
1.202741610010599e-15
0.0


In [54]:
eta = 0
eta = get_eta_coeffs_from_sample_matrix(A)
rev_eval = e_vals[::-1]
lre = 0
for i in range(len(eta)):
    lre += e_vals[i]*eta[i]
lre

In [55]:
get_eta_coeffs_from_sample_matrix2(A)

NameError: name 'get_eta_coeffs_from_sample_matrix2' is not defined

### Linear regression

In [56]:
noise_param = [point[:3] for point in data_points]
energies = e_vals
# Fit a linear regression model
model = LinearRegression()
model.fit(noise_param, energies)

# Extrapolate to (0, 0, 0)
extrapolated_value = model.predict([[0, 0, 0]]) # type: ignore

print("Extrapolated value:", extrapolated_value)

Extrapolated value: [-5.99302882]


In [57]:
n, d = 2, 2
sample_points = [(-3, 2), (0, 1), (-2, -1),  (-1, 2),  (1, 3), (2, 1) ]
monomials = get_monomials(n, d)

print(f"Monomials: {monomials}")
mat = sample_matrix(sample_points=sample_points, degree=d)
np.linalg.det(mat)

Monomials: ['1', 'λ_2', 'λ_1', 'λ_2**2', 'λ_1*λ_2', 'λ_1**2']


In [58]:
data_points = [

    [36, 9, 66, [-4.331378173828132]],
    [12, 5, 66, [-5.137601571314731]],
    [60, 15, 18, [-4.695641935081705]],
    [4, 1, 0, [-5.847083978239625]],
    [36, 9, 18, [-5.163901197964829]],
    [36, 9, 42, [-4.750859450731831]],
    [36, 9, 6, [-5.359889992461941]],
    [12, 3, 6, [-5.7587171216700765]],
    [12, 5, 42, [-5.381585427158724]],
    [20, 7, 18, [-5.366706940188518]]
]

noise_data = [tuple(point[:3]) for point in data_points]
e_vals = [point[-1][0] for point in data_points]

In [59]:
noise_data

In [60]:
n, d = 3, 2
sample_points = [ (12, 5, 66),(20, 7, 18), (36, 9, 66),(60, 15, 18), (4, 1, 0), (36, 9, 18), (36, 9, 42),  (12, 5, 42),  (36, 9, 6), (12, 3, 6),]
monomials = get_monomials(n, d)

print(f"Monomials: {monomials}")
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

Monomials: ['1', 'λ_3', 'λ_2', 'λ_1', 'λ_3**2', 'λ_2*λ_3', 'λ_2**2', 'λ_1*λ_3', 'λ_1*λ_2', 'λ_1**2']


⎡1.0  66.0  5.0   12.0  4356.0  330.0  25.0   792.0   60.0   144.0 ⎤
⎢                                                                  ⎥
⎢1.0  18.0  7.0   20.0  324.0   126.0  49.0   360.0   140.0  400.0 ⎥
⎢                                                                  ⎥
⎢1.0  66.0  9.0   36.0  4356.0  594.0  81.0   2376.0  324.0  1296.0⎥
⎢                                                                  ⎥
⎢1.0  18.0  15.0  60.0  324.0   270.0  225.0  1080.0  900.0  3600.0⎥
⎢                                                                  ⎥
⎢1.0   0    1.0   4.0     0       0     1.0     0      4.0    16.0 ⎥
⎢                                                                  ⎥
⎢1.0  18.0  9.0   36.0  324.0   162.0  81.0   648.0   324.0  1296.0⎥
⎢                                                                  ⎥
⎢1.0  42.0  9.0   36.0  1764.0  378.0  81.0   1512.0  324.0  1296.0⎥
⎢                                                                  ⎥
⎢1.0  42.0  5.0   12.0  1764.0  21

---

In [61]:
sample_points_new = [(4, 1), (12, 5), (28, 5), (44, 7), (20, 7), (44, 9)]
o_values = [-6.653015381479125, -5.944264589467563, -5.953575981631173, -5.872561179170413, -6.1660998573827825, -5.435931276419277]
n, d = 2, 2
mat = sample_matrix(sample_points=sample_points_new, degree=d)
eta_co = get_eta_coeffs_from_sample_matrix(mat)

In [62]:
matrices = generate_modified_matrices(mat)
detA = np.linalg.det(mat)
RE = 0
# Loop over coefficients and matrices
for E, matrix in zip(o_values, matrices):
    print(np.linalg.det(matrix) / detA)
    RE += E * (np.linalg.det(matrix) / detA )

RE

1.4615384615384595
-0.4423076923076924
-0.4519230769230766
0.15384615384615394
0.2884615384615387
-0.009615384615384089


In [63]:
eta_co

In [64]:
RE = 0
for i in range (len(o_values)):
    RE += (o_values[i])*eta_co[i]
RE

In [65]:
total = 0 
for a, b in zip(o_values, eta_co):
        total += a * b

In [66]:
total

In [67]:
Matrix(mat)

⎡1.0  1.0  4.0   1.0    4.0    16.0 ⎤
⎢                                   ⎥
⎢1.0  5.0  12.0  25.0  60.0   144.0 ⎥
⎢                                   ⎥
⎢1.0  5.0  28.0  25.0  140.0  784.0 ⎥
⎢                                   ⎥
⎢1.0  7.0  44.0  49.0  308.0  1936.0⎥
⎢                                   ⎥
⎢1.0  7.0  20.0  49.0  140.0  400.0 ⎥
⎢                                   ⎥
⎣1.0  9.0  44.0  81.0  396.0  1936.0⎦

In [68]:
n_rows, n_cols = mat.shape

In [69]:
from hmac import new


terms = []
for i in range(n_rows):
    new_mat = mat.copy()
    new_mat[i] = np.array([[0] * (n_cols - 1) + [1]])  
    terms.append(np.linalg.det(new_mat) / detA)

In [70]:
Matrix(new_mat)

⎡1.0  1.0  4.0   1.0    4.0    16.0 ⎤
⎢                                   ⎥
⎢1.0  5.0  12.0  25.0  60.0   144.0 ⎥
⎢                                   ⎥
⎢1.0  5.0  28.0  25.0  140.0  784.0 ⎥
⎢                                   ⎥
⎢1.0  7.0  44.0  49.0  308.0  1936.0⎥
⎢                                   ⎥
⎢1.0  7.0  20.0  49.0  140.0  400.0 ⎥
⎢                                   ⎥
⎣ 0    0    0     0      0     1.0  ⎦

In [71]:
Matrix(mat)

⎡1.0  1.0  4.0   1.0    4.0    16.0 ⎤
⎢                                   ⎥
⎢1.0  5.0  12.0  25.0  60.0   144.0 ⎥
⎢                                   ⎥
⎢1.0  5.0  28.0  25.0  140.0  784.0 ⎥
⎢                                   ⎥
⎢1.0  7.0  44.0  49.0  308.0  1936.0⎥
⎢                                   ⎥
⎢1.0  7.0  20.0  49.0  140.0  400.0 ⎥
⎢                                   ⎥
⎣1.0  9.0  44.0  81.0  396.0  1936.0⎦

---

In [72]:
n, d = 3, 2
detA = 0, 0

data_points = [
    
    [60, 15, 18, [-4.695641935081705]],
    [4, 1, 0, [-5.847083978239625]],
    [36, 9, 42, [-4.750859450731831]],
    
    [12, 5, 42, [-5.381585427158724]],
    [20, 7, 18, [-5.366706940188518]],
    [36, 9, 66, [-4.331378173828132]],
    [36, 9, 6, [-5.359889992461941]],
    [12, 3, 6, [-5.7587171216700765]],
    [12, 5, 66, [-5.137601571314731]],
    [36, 9, 18, [-5.163901197964829]]
    
]

sample_noise = [tuple(point[:3]) for point in data_points]
e_vals = [point[-1][0] for point in data_points]

samplematrix = sample_matrix(sample_points= sample_noise, degree=d)
eta_coeffi = get_eta_coeffs_from_sample_matrix(samplematrix)

extrapolated_value = 0
for a, b in zip(eta_coeffi, e_vals):
    if len(eta_coeffi) != len(e_vals ):
        raise ValueError("Lists must be of the same length.")
    extrapolated_value += a * b

print(extrapolated_value)


matrices = generate_modified_matrices(samplematrix)
detA = np.linalg.det(samplematrix)
RE = 0
# Loop over coefficients and matrices
for E, matrix in zip(e_vals, matrices):
    #print(np.linalg.det(matrix) / detA)
    RE += E * (np.linalg.det(matrix) / detA )
print (f"My RE is: {RE}")


noise_param = [point[:3] for point in data_points]
energies = e_vals
# Fit a linear regression model
model = LinearRegression()
model.fit(noise_param, energies)

# Extrapolate to (0, 0, 0)
linear_value = model.predict([[0, 0, 0]]) # type: ignore

print(f"Linerar Extrapolated value:{linear_value}")

-19040035839.929688
My RE is: 17.367657780334966
Linerar Extrapolated value:[-5.99302882]


---

In [73]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# nqubit = 5
# data_points = [

#     [4, 1, 0, [-5.866066884530623]],
#     [20, 3, 6, [-5.7875550813689856]],
#     [12, 5, 6, [-5.774697386578676]],
#     [36, 7, 6, [-5.586486292622952]],
#     [12, 9, 6, [-5.679030576950535]],
#     [20, 9, 6, [-5.602724878007255]]
# ]

# nqubit = 4
# Exact = -4.758770483143634
data_points = [

    [4, 1, 0, [-4.729146163428284]],
    [12, 3, 4, [-4.659161735860657]],
    [28, 5, 4, [-4.586098614568544]],
    [12, 7, 4, [-4.582238777333613]],
    [20, 9, 4, [-4.524032648531302]],
    [36, 5, 4, [-4.561422365560775]],
    
]

sample_points = [tuple(point[:2]) for point in data_points]
energies = [point[-1][0] for point in data_points]

n, d = 2, 2
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")

A = sample_matrix(sample_points= sample_points, degree= d)
coeffs = get_eta_coeffs_from_sample_matrix(A)

sum = 0
for e, c in zip(energies, coeffs):
    #print(e)
    sum += e*c

print(f"ZNE val1 = {sum}")

sum2 = 0
matrices = generate_modified_matrices(A)
for e, m in zip(energies, matrices):
    sum2 += e* ( (np.linalg.det(m)) / np.linalg.det(A) )
print(f"ZNE val2 = {sum2}")

# Generate polynomial features
poly = PolynomialFeatures(degree=d)
X_poly = poly.fit_transform([point[:2] for point in data_points])

# Fit linear regression model on polynomial features
model = LinearRegression()
model.fit(X_poly, energies)

# Define the point (0,0,0) for prediction
point_to_predict = np.array([[0, 0]])

# Transform the point into polynomial features
point_to_predict_poly = poly.transform(point_to_predict)

# Predict the energy at the point
predicted_energy = model.predict(point_to_predict_poly)

print("Sklearn Poly exttrapolation:", predicted_energy[0])

Monomials: ['1', 'λ_2', 'λ_1', 'λ_2**2', 'λ_1*λ_2', 'λ_1**2']
Monomials: 6
ZNE val1 = -2.992343480150844e-06
ZNE val2 = -4.77238109293036
Sklearn Poly exttrapolation: -4.772381092930383


### 3-variable

In [3]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# nqubit = 5
# Exact = -6.026674183332272
data_points = [

    
    [12, 5, 18, [-3.717054761326808]],
    [20, 1, 30, [-3.874319285474221]],
    [4, 5, 6, [-3.8215866373279432]],
    [28, 7, 6, [-3.687171847866942]],
    [4, 9, 18, [-3.5663020157522007]],
    [4, 1, 0, [-3.9308629429508755]],
    [12, 3, 6, [-3.84785786134227]],
    [4, 11, 6, [-3.6634401626134006]],
    [20, 11, 30, [-3.2597725821448074]],
    [36, 11, 42, [-3.053170929882078]]
 
]

# nqubit = 4
# Exact = -4.758770483143634
# data_points = [

#     [4, 1, 0, [-4.727223587006228]],
#     [12, 3, 12, [-4.607713830417823]],
#     [12, 5, 28, [-4.407043007625323]],
#     [4, 7, 4, [-4.576456763037398]],
#     [4, 7, 12, [-4.469395375908336]],
#     [12, 7, 36, [-4.1631178010204]],
#     [12, 9, 20, [-4.37387921098692]],
#     [20, 9, 4, [-4.454573543378764]],
#     [20, 1, 20, [-4.6775229831603005]],
#     [4, 13, 4, [-4.430628046997407]]

#     # [4, 1, 0, [-4.728411487887455]],
#     # [12, 3, 4, [-4.633252562558045]],
#     # [20, 3, 4, [-4.602476577557038]],
#     # [20, 5, 4, [-4.581231642506445]],
#     # [12, 7, 4, [-4.543444180800407]],
#     # [20, 7, 4, [-4.5123253066695685]],
#     # # [4, 9, 4, [-4.551672450645599]],
#     # [12, 11, 4, [-4.526468988653714]],
#     # [12, 3, 20, [-4.565851633863804]],
#     # [20, 11, 4, [-4.502139260733794]],
#     # [12, 5, 20, [-4.494436896236834]]
#     # # [28, 11, 4, [-4.477859709306907]],
#     # # # [36, 11, 4, [-4.4552360261829405]],
#     # # # [12, 13, 4, [-4.395126157842261]]
    
# ]

sample_points = [tuple(point[:3]) for point in data_points]
energies = [point[-1][0] for point in data_points]

n, d = 3, 2
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")

A = sample_matrix(sample_points= sample_points, degree= d)
coeffs = get_eta_coeffs_from_sample_matrix(A)

sum = 0
for e, c in zip(energies, coeffs):
    #print(e)
    sum += e*c

print(f"ZNE val1 = {sum}")

sum2 = 0
matrices = generate_modified_matrices(A)
for e, m in zip(energies, matrices):
    sum2 += e* ( (np.linalg.det(m)) / np.linalg.det(A) )
print(f"ZNE val2 = {sum2}")

# Generate polynomial features
poly = PolynomialFeatures(degree=d)
X_poly = poly.fit_transform([point[:3] for point in data_points])

# Fit linear regression model on polynomial features
model = LinearRegression()
model.fit(X_poly, energies)

# Define the point (0,0,0) for prediction
point_to_predict = np.array([[0, 0, 0]])

# Transform the point into polynomial features
point_to_predict_poly = poly.transform(point_to_predict)

# Predict the energy at the point
predicted_energy = model.predict(point_to_predict_poly)

print("Sklearn Poly exttrapolation:", predicted_energy[0])
print("Qubit = 5; layer = 10, Exact val = -3.999999999999999")

Monomials: ['1', 'λ_3', 'λ_2', 'λ_1', 'λ_3**2', 'λ_2*λ_3', 'λ_2**2', 'λ_1*λ_3', 'λ_1*λ_2', 'λ_1**2']
Monomials: 10
ZNE val1 = 3.5038020569726734e-05
ZNE val2 = -3.951970611035856
Sklearn Poly exttrapolation: -3.951970611035858
Qubit = 5; layer = 10, Exact val = -3.999999999999999
